<a href="https://colab.research.google.com/github/LCaravaggio/AnalisisPredictivo/blob/master/Kaggle/2023Q2/Un_primer_LR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import json,re
from sklearn.model_selection import train_test_split
from concurrent.futures import ThreadPoolExecutor
from preparando_datos import sum_into_column,split_and_sum

import pandas as pd
import matplotlib.pyplot as plt





In [19]:
df_val=pd.read_csv('dataset/testear.csv')
df_train=pd.read_csv('dataset/origen.csv')


In [20]:
# Seasons worked
for df in [df_train,df_val]:
    writers = df.groupby("directors").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["directors_exp"] = df["directors"].apply(sum_into_column,args=(writers,))
    
for df in [df_val,df_train]:
    seasons = df.groupby("directors").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["dir_seasonsOfExperience"] = df["directors"].apply(sum_into_column,args=(seasons,))

# episodes worked
for df in [df_val,df_train]:
    episodes = df.groupby("directors").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["dir_episodesOfExperience"] = df["directors"].apply(sum_into_column,args=(episodes,))
    
# movies worked
for df in [df_val,df_train]:
    movies = df[df["titleType"] == "movie"].groupby("directors").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["dir_moviesOfExperience"] = df["directors"].apply(sum_into_column,args=(movies,))

# other xp than movies
for df in [df_val,df_train]:
    others = df[df["titleType"] != "movie"].groupby("directors").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["dir_othersOfExperience"] = df["directors"].apply(sum_into_column,args=(others,))

del seasons,episodes,movies,others,df

In [21]:
for df in [df_train,df_val]:
    writers = df.groupby("writers").count()["Unnamed: 0"].to_dict()
    writers = split_and_sum(writers)
    df["writers_exp"] = df["writers"].apply(sum_into_column,args=(writers,))

    seasons = df.groupby("writers").count()["seasonNumber"].to_dict()
    seasons = split_and_sum(seasons)
    df["writer_seasonsOfExperience"] = df["writers"].apply(sum_into_column,args=(seasons,))

    episodes = df.groupby("writers").count()["episodeNumber"].to_dict()
    episodes = split_and_sum(episodes)
    df["writer_episodesOfExperience"] = df["writers"].apply(sum_into_column,args=(episodes,))

    movies = df[df["titleType"] == "movie"].groupby("writers").count()["titleType"].to_dict()
    movies = split_and_sum(movies)
    df["writer_moviesOfExperience"] = df["writers"].apply(sum_into_column,args=(movies,))

    others = df[df["titleType"] != "movie"].groupby("writers").count()["titleType"].to_dict()
    others = split_and_sum(others)
    df["writer_othersOfExperience"] = df["writers"].apply(sum_into_column,args=(others,))


del seasons,episodes,movies,others,df

In [22]:
def parse_json(x):
    x = x.split(":")
    x = x[1:]
    for i in range(len(x)):
        x[i] = x[i].split("}")[0]
        x[i] = x[i].split(",")[0]
    
    #split the list into pairs of two
    x = [x[i:i+2] for i in range(0,len(x),2)]
    # grab the first element of the pair
    x = [i[0] for i in x]
    #remove the quotes
    x = [i.replace("'",'') for i in x]
    #remove the spaces
    x = [i.strip() for i in x]

    return ",".join(x)
for df in [df_train,df_val]:
    #turn production company into a list
    #fill nans with empty list
    df["production_countries"] = df["production_countries"].fillna("[]")
    #turn str list into list
    df["production_countries"] = df["production_countries"].apply(parse_json)
    experience = split_and_sum(df.groupby("production_countries").count()["Unnamed: 0"].to_dict())
    experience['']=0
    df["production_countries_experience"] = df["production_countries"].apply(sum_into_column,args=(experience,))

    df['production_countries'] = df['production_countries'].apply(lambda x: len(x.split(',')))

    #turn production company into a list
    #fill nans with empty list
    df["production_companies"] = df["production_companies"].fillna("[]")
    #turn str list into list
    df["production_companies"] = df["production_companies"].apply(parse_json)
    experience = split_and_sum(df.groupby("production_companies").count()["Unnamed: 0"].to_dict())
    experience['']=0
    df["production_companies_experience"] = df["production_companies"].apply(sum_into_column,args=(experience,))

    df['production_companies'] = df['production_companies'].apply(lambda x: len(x.split(',')))
    dir_votes = df.groupby("directors").sum()["numVotes"].to_dict()
    dir_votes = split_and_sum(dir_votes)
    df["dir_votes"] = df["directors"].apply(sum_into_column,args=(dir_votes,))
    
    

C:\Users\ianda\AppData\Local\Temp\ipykernel_8904\4059846201.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dir_votes = df.groupby("directors").sum()["numVotes"].to_dict()
C:\Users\ianda\AppData\Local\Temp\ipykernel_8904\4059846201.py:40: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  dir_votes = df.groupby("directors").sum()["numVotes"].to_dict()


In [23]:
# turn video colunm into 1 or 0
# video column might have False, True or NaN values
for df in [df_val,df_train]:
    df["video"] = df["video"].fillna(False)
    df["video"] = df["video"].astype(int)

# turn tagline into the lenth of the tagline
for df in [df_val,df_train]:
    df["tagline"] = df["tagline"].fillna("")
    df["tagline"] = df["tagline"].apply(len)

In [24]:
#set language to 0 if it is nan

df_train['language'] = df_train['language'].fillna(0)
df_val['language'] = df_val['language'].fillna(0)

In [25]:
y=df_train.averageRating
vars=['startYear', 'runtimeMinutes',"numVotes","writers_exp",
      "directors_exp","endYear","isAdult","seasonNumber",
      "episodeNumber","ordering","isOriginalTitle","runtime",
      "revenue","popularity","budget","dir_seasonsOfExperience",
      "dir_episodesOfExperience","dir_othersOfExperience","dir_moviesOfExperience",
      "writer_seasonsOfExperience","writer_episodesOfExperience","writer_othersOfExperience",
      "writer_moviesOfExperience","video","tagline","production_countries","production_countries_experience",
      "dir_votes",'production_companies', 'production_companies_experience']

X=df_train[vars]
X_pred=df_val[vars]

for db in [[X,df_train],[X_pred,df_val]]:
      db[0] = pd.concat([db[0], pd.get_dummies(db[1]['language'], prefix='language')], axis=1)
      db[0] = pd.concat([db[0], pd.get_dummies(db[1]['status'], prefix='status')], axis=1)

del db



In [26]:
for i,df in enumerate([df_train, df_val]):
    encoder = OneHotEncoder()
    title = encoder.fit_transform(df[['titleType']])
    title_df = pd.DataFrame(title.toarray(), columns=encoder.get_feature_names_out(['titleType']))
    #split by genres
    genres = df.genres_x.str.get_dummies(sep=',')
    genres = genres.reindex(sorted(genres.columns), axis=1)

    title_df = pd.concat([title_df, genres], axis=1)
    if not i:
        X = pd.concat([X, title_df], axis=1)
    else:
        X_pred = pd.concat([X_pred, title_df], axis=1)
    print(i,"Done!")
    


del title_df, genres, title, encoder, df,i





0 Done!
1 Done!


In [27]:
#drop duplicate columns
X = X.loc[:,~X.columns.duplicated()]
X_pred = X_pred.loc[:,~X_pred.columns.duplicated()]
print(X.columns)
#add missing columns
for col in X.columns:
    if col not in X_pred.columns:
        X_pred[col] = 0
for col in X_pred.columns:
    if col not in X.columns:
        X[col] = 0

Index(['startYear', 'runtimeMinutes', 'numVotes', 'writers_exp',
       'directors_exp', 'endYear', 'isAdult', 'seasonNumber', 'episodeNumber',
       'ordering', 'isOriginalTitle', 'runtime', 'revenue', 'popularity',
       'budget', 'dir_seasonsOfExperience', 'dir_episodesOfExperience',
       'dir_othersOfExperience', 'dir_moviesOfExperience',
       'writer_seasonsOfExperience', 'writer_episodesOfExperience',
       'writer_othersOfExperience', 'writer_moviesOfExperience', 'video',
       'tagline', 'production_countries', 'production_countries_experience',
       'dir_votes', 'production_companies', 'production_companies_experience',
       'titleType_movie', 'titleType_short', 'titleType_tvEpisode',
       'titleType_tvMiniSeries', 'titleType_tvMovie', 'titleType_tvSeries',
       'titleType_tvShort', 'titleType_tvSpecial', 'titleType_video',
       'titleType_videoGame', '0', 'Action', 'Adult', 'Adventure', 'Animation',
       'Biography', 'Comedy', 'Crime', 'Documentary', 'Dram

In [28]:
X_pred = X_pred.sort_index(axis=1)
X = X.sort_index(axis=1)
X.to_csv('dataset/train.csv', index=False)
X_pred.to_csv('dataset/val.csv', index=False)